In [2]:
import splitfolders, os

# C:\ProjectWork\Basic_Python\Zero_shot_Learning\Dataset\CUB_KMean_Dataset\ViT_data

row_dataset_path = os.path.join('..', 'Dataset', 'CUB_KMean_Dataset', 'images')
split_dataset_path = os.path.join('..', 'Dataset', 'data', 'CUB_data', 'images')

splitfolders.ratio(row_dataset_path, split_dataset_path, seed=1111, ratio=(0.7, 0.15, 0.15))

Copying files: 5939 files [00:19, 309.54 files/s]


In [ ]:
# libs
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Dropout


In [57]:
# variables

# Set dataset paths
train_dir = os.path.join(split_dataset_path, 'train')
val_dir = os.path.join(split_dataset_path, 'val') 
test_dir = os.path.join(split_dataset_path, 'test') 
num_classes = len(os.listdir(train_dir))

# Image dimensions
img_size = (224, 224)  # Resize to 128x128
batch_size = 1
latent_dim = 16  # Dimension of latent space



In [60]:

# Load dataset
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(train_dir, 
                                              target_size=img_size, 
                                              batch_size=batch_size)

val_generator = datagen.flow_from_directory(val_dir, 
                                            target_size=img_size, 
                                            batch_size=batch_size)

test_generator = datagen.flow_from_directory(test_dir, 
                                             target_size=img_size, 
                                             batch_size=batch_size)


Found 4146 images belonging to 101 classes.
Found 872 images belonging to 101 classes.
Found 921 images belonging to 101 classes.


In [70]:
# # Autoencoder Model
# input_img = Input(shape=(224, 224, 3))
# input_shape = input_img.shape[1]

# # Sample Test 1
# # Encoder layers
# encoded = Dense(32, activation='relu')(input_img)
# encoded = Dense(latent_dim, activation='relu')(encoded)

# # Decoder layers
# decoded = Dense(32, activation='relu')(encoded)
# decoded = Dense(input_shape, activation='sigmoid')(decoded) # num_classes

# # Compile model
# autoencoder = Model(input_img, decoded)
# autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')


# # Set steps_per_epoch correctly
# steps_per_epoch = int(train_generator.samples // batch_size)
# validation_steps = int(val_generator.samples // batch_size)

# # Train model
# autoencoder.fit(train_autoencoder_gen, 
#                 validation_data=val_autoencoder_gen, 
#                 epochs=10, 
#                 steps_per_epoch=steps_per_epoch, 
#                 validation_steps=validation_steps)

# # Save model
# autoencoder.save("autoencoder.h5")



In [ ]:
# # Load and predict
# autoencoder = tf.keras.models.load_model("autoencoder.h5")
# sample_images = next(test_generator)[:5]  # Get sample images
# reconstructed_images = autoencoder.predict(sample_images)

# # Display results
# import matplotlib.pyplot as plt

# fig, axes = plt.subplots(2, 5, figsize=(10, 4))
# for i in range(5):
#     axes[0, i].imshow(sample_images[i])
#     axes[0, i].axis('off')
#     axes[1, i].imshow(reconstructed_images[i])
#     axes[1, i].axis('off')
# plt.show()

In [62]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [63]:
def build_autoencoder(input_shape):
    # Encoder
    encoder = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same')
    ])

    # Decoder
    decoder = models.Sequential([
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')  # Assuming 3-channel images
    ])

    # Autoencoder
    autoencoder = models.Sequential([encoder, decoder])
    return autoencoder, encoder

In [64]:
# Set dataset paths
train_dir = os.path.join(split_dataset_path, 'train')
val_dir = os.path.join(split_dataset_path, 'val') 
test_dir = os.path.join(split_dataset_path, 'test') 
num_classes = len(os.listdir(train_dir))

# Image dimensions
img_height, img_width = 224, 224
batch_size = 16

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='input'  # For autoencoder, we use 'input' as target is the same as input
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='input'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='input'
)

Found 4146 images belonging to 101 classes.
Found 872 images belonging to 101 classes.
Found 921 images belonging to 101 classes.


In [66]:
input_shape = (img_height, img_width, 3)
autoencoder, encoder = build_autoencoder(input_shape)

autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train the autoencoder
autoencoder.fit(
    train_generator,
    validation_data=val_generator,
    epochs=1,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=val_generator.samples // batch_size
)


259/259 [==============================] - 482s 2s/step - loss: 0.0144 - accuracy: 0.6617 - val_loss: 0.0080 - val_accuracy: 0.7735


In [67]:
def extract_features(generator, encoder):
    features = []
    labels = []
    for i in range(len(generator)):
        x, _ = generator[i]  # Discard the target (input) for feature extraction
        encoded_features = encoder.predict(x)
        features.append(encoded_features)
        labels.append(generator.classes[generator.index_array[i*batch_size:(i+1)*batch_size]])
    return np.vstack(features), np.hstack(labels)

train_features, train_labels = extract_features(train_generator, encoder)
val_features, val_labels = extract_features(val_generator, encoder)
test_features, test_labels = extract_features(test_generator, encoder)

1/1 [==============================] - 0s 165ms/step


1/1 [==============================] - 0s 171ms/step


1/1 [==============================] - 0s 119ms/step


In [68]:
num_classes = 101

# Define classifier
classifier = models.Sequential([
    layers.Input(shape=train_features.shape[1:]),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train classifier
classifier.fit(
    train_features, train_labels,
    validation_data=(val_features, val_labels),
    epochs=30,
    batch_size=batch_size
)

Epoch 1/30
260/260 [==============================] - 162s 505ms/step - loss: 4.7322 - accuracy: 0.0080 - val_loss: 4.6148 - val_accuracy: 0.0103
Epoch 2/30
260/260 [==============================] - 100s 383ms/step - loss: 4.6170 - accuracy: 0.0068 - val_loss: 4.6143 - val_accuracy: 0.0103
Epoch 3/30
260/260 [==============================] - 107s 412ms/step - loss: 4.6176 - accuracy: 0.0089 - val_loss: 4.6139 - val_accuracy: 0.0103
Epoch 4/30
260/260 [==============================] - 101s 390ms/step - loss: 4.6142 - accuracy: 0.0092 - val_loss: 4.6128 - val_accuracy: 0.0126
Epoch 5/30
260/260 [==============================] - 98s 376ms/step - loss: 4.6186 - accuracy: 0.0063 - val_loss: 4.6134 - val_accuracy: 0.0103
Epoch 6/30
260/260 [==============================] - 90s 348ms/step - loss: 4.6143 - accuracy: 0.0084 - val_loss: 4.6132 - val_accuracy: 0.0103
Epoch 7/30
260/260 [==============================] - 97s 374ms/step - loss: 4.6136 - accuracy: 0.0082 - val_loss: 4.6131 - va

In [69]:
classifier.save('classification_model.h5')
encoder.save('encoder_model.h5')

C:\ProjectWork\Basic_Python\Zero_shot_Learning\venv_zsl\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [78]:
from tensorflow.keras.preprocessing import image

def classify_single_image(image_path, encoder, classifier):
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Extract features using encoder
    encoded_features = encoder.predict(img_array)

    # Predict class
    predictions = classifier.predict(encoded_features)
    predicted_class = np.argmax(predictions, axis=1)[0]
    
    return predicted_class



# Example usage
image_path = os.path.join(train_dir, 'Acadian_Flycatcher', 'Acadian_Flycatcher_0003_29094.jpg')
predicted_class = classify_single_image(image_path, encoder, classifier)
print(f"Predicted Class: {predicted_class}")

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir)
class_names = train_dataset.class_names
predicted_class_name = class_names[predicted_class]
print(f"Predicted Class Name: {predicted_class_name}")

1/1 [==============================] - 0s 32ms/step
Predicted Class: 9
Found 4146 files belonging to 101 classes.
Predicted Class Name: Barn_Swallow
